In [1]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Data Pre-processing

In [3]:
""" Get all the notes and chords from the midi files in the ./midi_songs directory """
notes = []

for file in glob.glob("/content/drive/MyDrive/lofi-music-generator/songs/*.mid*"):
    midi = converter.parse(file)
    
    print("Parsing %s" % file)
    
    notes_to_parse = None
    
    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse()
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
        
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

with open('/content/drive/MyDrive/lofi-music-generator/data/notes', 'wb') as filepath:
    pickle.dump(notes, filepath)

Parsing /content/drive/MyDrive/lofi-music-generator/songs/figaro.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/LofiPianoSample8.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/thenightmarebegins.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/Life_Stream.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/rufus.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/Zelda_Overworld.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/JENOVA.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/LofiPianoSample9.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/FF4.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/ff4-fight1.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/BlueStone_LastDungeon.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/ff8-lfp.mid
Parsing /content/drive/MyDrive/lofi-music-generator/songs/0fithos.mid
Parsing /content/drive/MyDrive/lofi-musi

In [4]:
""" Prepare the sequences used by the Neural Network """
n_vocab = len(set(notes)) 

sequence_length = 32

# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length]
  sequence_out = notes[i + sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  network_output.append(note_to_int[sequence_out])
    
  n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

# normalize input
network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

# 2. LSTM Neural Network

In [5]:
""" Create the structure of the neural network """
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [6]:
""" Create Model Checkpoint """
filepath = "/content/drive/MyDrive/lofi-music-generator/training_weights/lofi-music-weights-1{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

In [7]:
""" Summary the neural network """
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32, 256)           264192    
_________________________________________________________________
dropout (Dropout)            (None, 32, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32, 512)           1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [ ]:
""" Train the neural network """
model.fit(network_input, network_output, epochs=100, batch_size=32, callbacks=callbacks_list)

Epoch 1/100
1849/1849 [==============================] - 45s 15ms/step - loss: 4.7699
Epoch 2/100
1565/1849 [========================>.....] - ETA: 4s - loss: 4.5721

# 3. Generate Music Melody

In [ ]:
""" Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []


# generate 300 notes
for note_index in range(300):
  prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))

  prediction = model.predict(prediction_input, verbose=0)

  index = numpy.argmax(prediction)
    
  result = int_to_note[index]
  prediction_output.append(result)    

  pattern = pattern[1:]*n_vocab
  pattern = numpy.append(pattern, index)
  pattern = pattern / float(n_vocab)

# 4. Create Music MIDI File

In [ ]:
""" convert the output from the prediction to notes and create a midi file from the notes """
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='output.mid')